<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Labo-App_Mobile/IdEcorces-ResConv-ConversionVersTFLite-Colab.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.


SVP, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

#  Conversion vers un modèle TensorFlow Lite

Comme son nom l'indique, le convertisseur TensorFlow Lite, <code>tf.lite.TFLiteConverter</code>, convertit un modèle TensorFlow Keras en un modèle TensorFlow Lite plus compact. Identifié par l'extension de fichier .tflite, le format TensorFlow Lite.



## Installation des bibliothèques Python

In [ ]:
# !pip3 install --upgrade tensorflow

In [ ]:
import os

import tensorflow as tf
print("Version de TensorFlow :",tf.__version__)

print("Bibliothèques Python chargées")

## Création des répertoires

In [ ]:
try:
    os.mkdir("/content/donnees/")
except OSError:
    pass
try:
    os.mkdir("/content/lab_ecorces/")
except OSError:
    pass
try:
    os.mkdir("/content/modele_keras/")
except OSError:
    pass
try:
    os.mkdir("/content/modele_TFlite")
except OSError:
    pass

## Téléversement et décompression d'un modèle Keras sauvegardé sur un poste local

1. & 2. Vous allez téléverser le modèle Keras entraîné précédemment.

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>

3. La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher le modèle Keras compressé, modele_....zip que vous avez sauvegardé précédemment sur votre ordinateur local.

Attention! Le téléchargement peut prendre plusieurs minutes. Assurez-vous que le fichier est entièrement téléversé et que l'icône d'état du téléversement (un cercle jaune, comme ci-dessous) disparaisse. 

<img style="margin-left:0px" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier-3.png"/>


4. Décompressez le fichier modele_....zip en exécutant la commande ci-dessous:<br/>

In [ ]:
!unzip /content/modele_*.zip -d /content/modele_keras && rm /content/modele_*.zip

In [ ]:
# Chemin vers modèles 
chemin_vers_modele_keras = "/content/modele_keras/"
# Taille du modèle Keras
taille_totale = 0
for chemin, repertoires, fichiers in os.walk(chemin_vers_modele_keras):
    for fichier in fichiers:
        chemin_fichier = os.path.join(chemin, fichier)
        taille_totale += os.path.getsize(chemin_fichier)
taille_modele_keras = taille_totale
print("Taille du modèle Keras: " + str(taille_modele_keras//1.0E+06) + " Mo")

## Chargement du modèle

In [ ]:
modele_keras = tf.keras.models.load_model(chemin_vers_modele_keras)

print("Architecture du modèle entraîné")
modele_keras.summary()


##  Conversion du modèle

L'examen d'un échantillon représentatif des données est requis pour réaliser une quantification (<i>quantization</i>), en fait une réduction de la précision des calculs avec des entiers à 8 bits (INT8 ou TFLITE_BUILTINS_INT8).

### Chargement des données

#### Utilisation de l'IPA (<i>API</i>) de Kaggle pour l'importation directe du jeu de données BarkNet

1. Commencez par installer la bibliothèque Python `kaggle`

In [ ]:
!pip3 install kaggle

2. Si ce n'est déjà fait, devenez membre de Kaggle avec votre adresse de courriel GMail:<br/>

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-1.png"/>

3. Maintenant, vous devez télécharger votre clé privée pour utiliser l'IPA de Kaggle.

4. Cliquez sur l'onglet « account » de votre profil Kaggle

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-2.png"/>

5. Sur la page « Account » cliquez sur le bouton « Create New API Token ».
    
<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-3.png"/>

6. Téléchargez votre clé privée « kaggle.json » pour l'IPA Kaggle dans un endroit temporaire sur votre poste de travail.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-5.png"/>

7. Maintenant, transférez (téléversez) votre clé privée « kaggle.json » dans votre environnement Colab.

La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher votre clé privée « kaggle.json » que vous avez sauvegardée sur votre  ordinateur.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>


8. Créer à la racine un répertoire .kaggle et déplacez votre clé privée « kaggle.json » dans ce répertoire.

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle -all

9. Maintenant téléchargez le jeu de données réduit « barknet-mini » de 1.6 Go avec la commande suivante:

In [ ]:
# Attention! Jeu réduit de données 1.5 Go - plus rapide à télécharger et à traiter
!kaggle datasets download claudecoulombe/barknet-mini --unzip -p /content/donnees/
repertoire_entree = "/content/donnees/BarkNet-mini"

### Répartition des données

In [ ]:
# Installation des bibliothèques Python `split-folders` et `tqdm`
!pip3 install split-folders tqdm

In [ ]:
# Répartition des données d'entraînement, de validation et de tests
import splitfolders
import pathlib

#### répertoire des données une fois réparties
repertoire_donnees_reparties = "/content/lab_ecorces"
# => train, val, test

nombre_images = len(list(pathlib.Path(repertoire_entree).glob('*/*.jpg')))
print("Nombre total d'images:",nombre_images)

splitfolders.ratio(repertoire_entree, 
                   output=repertoire_donnees_reparties, 
                   seed=42, 
                   ratio = (0.65, 0.15, 0.20)
                   )

print("\nRépartition des données terminée!")

## Conversion du modèle proprement dite

In [ ]:
TAILLE_LOT = 32
HAUTEUR_IMAGE = 150
LARGEUR_IMAGE = 150
TAILLE_IMAGE = (HAUTEUR_IMAGE, LARGEUR_IMAGE)
REPERTOIRE_VALIDATION = "/content/lab_ecorces/val"
chemin_vers_modele_TFlite = "/content/modele_TFlite/"
nom_modele_TFlite = "modele.tflite"

In [ ]:
# Source: Convert Keras models to TensorFlow Lite - David Cochard - 2 juin 2021
# https://medium.com/axinc-ai/convert-keras-models-to-tensorflow-lite-e654994fb93c

import numpy as np
import cv2
import glob
import sys

image_folder = REPERTOIRE_VALIDATION
img_path = glob.glob(image_folder+"/*/*")
if len(img_path)==0:
    print("Images absentes")
    sys.exit(1)
else:
    print("Images présentes")

validation_data_set=[]
for file_name in img_path:
    img = cv2.imread(file_name) #BGR
    img = cv2.resize(img,(150, 150))
    ary = np.asarray(img, dtype=np.float32)
    ary = np.expand_dims(ary, axis=0)
    ary = ary/255.0
    validation_data_set.append(ary)

def representative_dataset_gen():
    for i in range(len(validation_data_set)):
        yield [validation_data_set[i]]

print("Code de génération prêt")

In [ ]:
# Conversion et quantification
convertisseur = tf.lite.TFLiteConverter.from_keras_model(modele_keras) 
convertisseur.optimizations = [tf.lite.Optimize.DEFAULT]
convertisseur.representative_dataset = representative_dataset_gen
convertisseur.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
convertisseur.inference_input_type = tf.uint8
convertisseur.inference_output_type = tf.uint8
modele_tflite = convertisseur.convert()

# Sauvegarde du modèle TFlite
with open(chemin_vers_modele_TFlite+nom_modele_TFlite, 'wb') as fichier:
    fichier.write(modele_tflite)
    
print("Conversion terminée!")

In [ ]:
# Taille du modèle TFlite
taille_modele_TFlite = os.path.getsize(chemin_vers_modele_TFlite+nom_modele_TFlite)
print("La taille du fichier compact TFlite: ",str(taille_modele_TFlite//1.0E+06) + " Mo")

## Télécharger le modèle TFlite sur son poste local

In [ ]:
from google.colab import files

files.download(chemin_vers_modele_TFlite+nom_modele_TFlite)

In [ ]:
print("Fin de l'exécution du carnet IPython")